In [1]:
# Import packages
import pandas as pd
from sqlalchemy import create_engine
import json
import os

pd.set_option('display.max_rows', 200)

In [2]:
%load_ext dotenv
%dotenv ../.env

In [3]:
con = create_engine(os.getenv('BUILD_ENGINE').replace('localhost', os.getenv('IP')))

In [4]:
# Build funtion to get the domains associated with a specific field
def get_domain(field, domain_path):
    with open(domain_path, "r") as domain:
        domain = json.load(domain)
    records = [i for i in domain['fields'] if i['field'] == field]
    if len(records) == 0: 
        print(f'cannot find field: {field} in domain table')
    else: 
        return records[0]['domain']

In [5]:
# Example
proxcode_domain = get_domain('proxcode', 'pluto_domains.json')
print(proxcode_domain)

{'0': 'Not Available', '1': 'Detached', '2': 'Semi-Attached', '3': 'Attached'}


In [6]:
# Example
proxcode_domain.keys()

dict_keys(['0', '1', '2', '3'])

In [7]:
# Specify latest PLUTO version
pluto = 'pluto'
# Specify previous PLUTO version or the one you want to compare to
plutoprev = 'dcp_pluto'

### Value check for the thirteen fields with expected domain values

##### includes summary statistics of unique value comparisions for the two PLUTO version, and lists of values that do not agree w/ domain list

### Zoning districts

In [8]:
sql = f'''
    WITH newcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {pluto})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist),
    prevcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {plutoprev})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist)
    SELECT a.zonedist, b.zonedist as zonedistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.zonedist = b.zonedist
    ORDER BY diff DESC;
'''

In [9]:
df = pd.read_sql(sql=sql, con=con)

In [10]:
df[df['diff'] != 0]

,zonedist,zonedistprev,countnew,countprev,diff
0,None,None,2561645.0,NaN,NaN
1,None,None,NaN,2560881.0,NaN
2,C1-9A,C1-9A,137.0,134.0,3.0
114,C4-3,C4-3,1852.0,1853.0,-1.0
115,R5B,R5B,31642.0,31643.0,-1.0
116,M1-5B,M1-5B,592.0,593.0,-1.0
117,M1-1/R7-2,M1-1/R7-2,189.0,190.0,-1.0
118,C1-9,C1-9,999.0,1000.0,-1.0
119,R3A,R3A,54836.0,54837.0,-1.0
120,C6-6,C6-6,281.0,282.0,-1.0


### Commercial overlay

In [11]:
sql = f'''
    WITH newcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {pluto}
    UNION ALL
    SELECT overlay2 as overlay FROM {pluto}
    )
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay),
    prevcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {plutoprev}
    UNION ALL
    SELECT overlay2 as overlay FROM {plutoprev})
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay)
    SELECT a.overlay, b.overlay as overlayprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.overlay = b.overlay
    ORDER BY a.overlay;
'''

In [12]:
df = pd.read_sql(sql=sql, con=con)

In [13]:
df

,overlay,overlayprev,countnew,countprev,diff
0,C1-1,C1-1,1587.0,1587.0,0.0
1,C1-2,C1-2,11031.0,11031.0,0.0
2,C1-3,C1-3,13786.0,13790.0,-4.0
3,C1-4,C1-4,8095.0,8097.0,-2.0
4,C1-5,C1-5,2540.0,2547.0,-7.0
5,C2-1,C2-1,1684.0,1684.0,0.0
6,C2-2,C2-2,6023.0,6023.0,0.0
7,C2-3,C2-3,11942.0,11952.0,-10.0
8,C2-4,C2-4,16714.0,16727.0,-13.0
9,C2-5,C2-5,1523.0,1523.0,0.0


### Special purpose district

In [14]:
sql = f'''
    WITH newcount as (
    WITH sdall as (
    SELECT spdist1 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist2 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist3 as spdist FROM {pluto}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist),
    prevcount as (
     WITH sdall as (
    SELECT spdist1 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist2 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist3 as spdist FROM {plutoprev}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist)
    SELECT a.spdist, b.spdist as spdistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.spdist = b.spdist
    ORDER BY diff DESC;
'''

In [15]:
df = pd.read_sql(sql=sql, con=con)

In [16]:
df[df['diff'] != 0]

,spdist,spdistprev,countnew,countprev,diff
0,None,None,2477048.0,NaN,NaN
1,None,None,NaN,2476583.0,NaN
64,LI,LI,550.0,551.0,-1.0
65,EC-6,EC-6,409.0,410.0,-1.0
66,MX-7,MX-7,189.0,190.0,-1.0
67,HY,HY,288.0,289.0,-1.0
68,EC-5,EC-5,558.0,559.0,-1.0
69,MX-6,MX-6,11.0,12.0,-1.0
70,OP,OP,13442.0,13443.0,-1.0
71,CL,CL,1333.0,1334.0,-1.0


In [17]:
#Check spdist domain
unique_spdist = df.spdist.tolist()
spdist_domain = list(get_domain('spdist1', 'pluto_domains.json').keys())

In [18]:
# in domain not in pluto:
[i for i in spdist_domain if i not in unique_spdist]

['MX-3']

In [19]:
# in pluto not in domain:
[i for i in unique_spdist if i not in spdist_domain]

[None, None, 'MX-20', 'MX-16/EC-6', 'MX-16/EC-5', 'BSC', 'SW']

### Limited height

In [20]:
sql = f'''
    WITH newcount as (
    SELECT ltdheight, count(*)
    FROM {pluto}
    GROUP BY ltdheight),
    prevcount as (
    SELECT ltdheight, count(*)
    FROM {plutoprev}
    GROUP BY ltdheight)
    SELECT a.ltdheight, b.ltdheight as ltdheightprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ltdheight = b.ltdheight
    ORDER BY a.ltdheight;
'''

In [21]:
df = pd.read_sql(sql=sql, con=con)

In [22]:
df

,ltdheight,ltdheightprev,countnew,countprev,diff
0,LH-1,LH-1,2239.0,2239.0,0.0
1,LH-1A,LH-1A,797.0,797.0,0.0
2,None,None,856566.0,NaN,NaN
3,None,None,NaN,856423.0,NaN


In [23]:
#Check ltdheight domain
unique_ltdheight = df.ltdheight.tolist()
ltdheight_domain = list(get_domain('ltdheight', 'pluto_domains.json').keys())

In [24]:
# in domain not in pluto:
[i for i in ltdheight_domain if i not in unique_ltdheight]

['LH-2', 'LH-3']

In [25]:
# in pluto not in domain:
[i for i in unique_ltdheight if i not in ltdheight_domain]

[None, None]

### Building class

In [26]:
sql = f'''
    WITH newcount as (
    SELECT bldgclass, count(*)
    FROM {pluto}
    GROUP BY bldgclass),
    prevcount as (
    SELECT bldgclass, count(*)
    FROM {plutoprev}
    GROUP BY bldgclass)
    SELECT a.bldgclass, b.bldgclass as bldgclassprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bldgclass = b.bldgclass
    ORDER BY diff DESC;
'''

In [27]:
df = pd.read_sql(sql=sql, con=con)

In [28]:
df[df['diff'] != 0]

,bldgclass,bldgclassprev,countnew,countprev,diff
0,None,None,616.0,NaN,NaN
1,None,None,NaN,473.0,NaN
2,Z7,Z7,167.0,109.0,58.0
204,D7,D7,2118.0,2119.0,-1.0
205,Q2,Q2,591.0,592.0,-1.0
206,D3,D3,1306.0,1307.0,-1.0
207,U0,U0,85.0,86.0,-1.0
208,Q7,Q7,27.0,28.0,-1.0
209,A9,A9,22752.0,22753.0,-1.0
210,Q9,Q9,284.0,285.0,-1.0


In [29]:
#Check bldgclass domain
unique_bldgclass = df.bldgclass.tolist()
bldgclass_domain = list(get_domain('bldgclass', 'pluto_domains.json').keys())

In [30]:
# in domain not in pluto:
[i for i in bldgclass_domain if i not in unique_bldgclass]

['A',
 'B',
 'C',
 'D',
 'E',
 'E3',
 'E4',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'R7',
 'S',
 'T',
 'U',
 'U3',
 'V',
 'W',
 'Y',
 'Z',
 'Z6']

In [31]:
# in pluto not in domain:
[i for i in unique_bldgclass if i not in bldgclass_domain]

[None, None]

### Land use

In [32]:
sql = f'''
    WITH newcount as (
    SELECT landuse, count(*)
    FROM {pluto}
    GROUP BY landuse),
    prevcount as (
    SELECT landuse, count(*)
    FROM {plutoprev}
    GROUP BY landuse)
    SELECT a.landuse, b.landuse as landuseprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.landuse = b.landuse
    ORDER BY a.landuse;
'''

In [33]:
df = pd.read_sql(sql=sql, con=con)

In [34]:
df[df['diff'] != 0]

,landuse,landuseprev,countnew,countprev,diff
0,01,01,565355.0,565356.0,-1.0
2,03,03,12845.0,12846.0,-1.0
3,04,04,54507.0,54508.0,-1.0
6,07,07,6308.0,6335.0,-27.0
8,09,09,4010.0,4017.0,-7.0
10,11,11,27694.0,27715.0,-21.0
11,None,None,3125.0,NaN,NaN
12,None,None,NaN,2924.0,NaN


In [35]:
#Check landuse domain
unique_landuse = df.landuse.tolist()
landuse_domain = list(get_domain('landuse', 'pluto_domains.json').keys())

In [36]:
# in domain not in pluto:
[i for i in landuse_domain if i not in unique_landuse]

[]

In [37]:
# in pluto not in domain:
[i for i in unique_landuse if i not in landuse_domain]

[None, None]

### Owner type

In [38]:
sql = f'''
    WITH newcount as (
    SELECT ownertype, count(*)
    FROM {pluto}
    GROUP BY ownertype),
    prevcount as (
    SELECT ownertype, count(*)
    FROM {plutoprev}
    GROUP BY ownertype)
    SELECT a.ownertype, b.ownertype as ownertypeprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ownertype = b.ownertype
    ORDER BY a.ownertype;
'''

In [39]:
df = pd.read_sql(sql=sql, con=con)

In [40]:
df

,ownertype,ownertypeprev,countnew,countprev,diff
0,C,C,11654.0,11654.0,0.0
1,M,M,80.0,80.0,0.0
2,O,O,1373.0,1373.0,0.0
3,P,P,522.0,522.0,0.0
4,X,X,17414.0,17414.0,0.0
5,None,None,828559.0,NaN,NaN
6,None,None,NaN,828416.0,NaN


In [41]:
#Check ownertype domain
unique_ownertype = df.ownertype.tolist()
ownertype_domain = list(get_domain('ownertype', 'pluto_domains.json').keys())

In [42]:
# in domain not in pluto:
[i for i in ownertype_domain if i not in unique_ownertype]

['NaN']

In [43]:
# in pluto not in domain:
[i for i in unique_ownertype if i not in ownertype_domain]

[None, None]

### Area source

In [44]:
sql = f'''
    WITH newcount as (
    SELECT areasource, count(*)
    FROM {pluto}
    GROUP BY areasource),
    prevcount as (
    SELECT areasource, count(*)
    FROM {plutoprev}
    GROUP BY areasource)
    SELECT a.areasource, b.areasource as areasourceprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.areasource = b.areasource
    ORDER BY a.areasource;
'''

In [45]:
df = pd.read_sql(sql=sql, con=con)

In [46]:
df

,areasource,areasourceprev,countnew,countprev,diff
0,0,0,139.0,136.0,3.0
1,2,2,813263.0,813263.0,0.0
2,4,4,179.0,182.0,-3.0
3,5,5,63.0,63.0,0.0
4,7,7,45342.0,45342.0,0.0
5,None,None,616.0,NaN,NaN
6,None,None,NaN,473.0,NaN


In [47]:
#Check areasource domain
unique_areasource = df.areasource.tolist()
areasource_domain = list(get_domain('areasource', 'pluto_domains.json').keys())

In [48]:
# in domain not in pluto:
[i for i in areasource_domain if i not in unique_areasource]

[]

In [49]:
# in pluto not in domain:
[i for i in unique_areasource if i not in areasource_domain]

[None, None]

### Extension

In [50]:
sql = f'''
    WITH newcount as (
    SELECT ext, count(*)
    FROM {pluto}
    GROUP BY ext),
    prevcount as (
    SELECT ext, count(*)
    FROM {plutoprev}
    GROUP BY ext)
    SELECT a.ext, b.ext as extprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ext = b.ext
    ORDER BY a.ext;
'''

In [51]:
df = pd.read_sql(sql=sql, con=con)

In [52]:
df

,ext,extprev,countnew,countprev,diff
0,E,E,48335.0,48335.0,0.0
1,EG,EG,36709.0,36709.0,0.0
2,G,G,264413.0,264413.0,0.0
3,N,N,451333.0,451333.0,0.0
4,None,None,58812.0,NaN,NaN
5,None,None,NaN,58669.0,NaN


In [53]:
#Check extension domain
unique_extension = df.ext.tolist()
extension_domain = list(get_domain('ext', 'pluto_domains.json').keys())

In [54]:
# in domain not in pluto:
[i for i in extension_domain if i not in unique_extension]

[]

In [55]:
# in pluto not in domain:
[i for i in unique_extension if i not in extension_domain]

['N', None, None]

### Proxy code

In [56]:
sql = f'''
    WITH newcount as (
    SELECT proxcode, count(*)
    FROM {pluto}
    GROUP BY proxcode),
    prevcount as (
    SELECT proxcode, count(*)
    FROM {plutoprev}
    GROUP BY proxcode)
    SELECT a.proxcode, b.proxcode as proxcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.proxcode = b.proxcode
    ORDER BY a.proxcode;
'''

In [57]:
df = pd.read_sql(sql=sql, con=con)

In [58]:
df

,proxcode,proxcodeprev,countnew,countprev,diff
0,0,0,144832.0,144832.0,0.0
1,1,1,313731.0,313731.0,0.0
2,2,2,199995.0,199995.0,0.0
3,3,3,200428.0,200428.0,0.0
4,None,None,616.0,NaN,NaN
5,None,None,NaN,473.0,NaN


In [59]:
#Check proxcode domain
unique_proxcode = df.proxcode.tolist()
proxcode_domain = list(get_domain('proxcode', 'pluto_domains.json').keys())

In [60]:
# in domain not in pluto:
[i for i in proxcode_domain if i not in unique_proxcode]

[]

In [61]:
# in pluto not in domain:
[i for i in unique_proxcode if i not in proxcode_domain]

[None, None]

### Lot type

In [62]:
sql = f'''
    WITH newcount as (
    SELECT lottype, count(*)
    FROM {pluto}
    GROUP BY lottype),
    prevcount as (
    SELECT lottype, count(*)
    FROM {plutoprev}
    GROUP BY lottype)
    SELECT a.lottype, b.lottype as lottypeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.lottype = b.lottype
    ORDER BY a.lottype;
'''

In [63]:
df = pd.read_sql(sql=sql, con=con)

In [64]:
df

,lottype,lottypeprev,countnew,countprev,diff
0,0,0,8534.0,8534.0,0.0
1,1,1,1454.0,1456.0,-2.0
2,2,2,1823.0,1827.0,-4.0
3,3,3,100674.0,100674.0,0.0
4,4,4,3443.0,3438.0,5.0
5,5,5,738045.0,738045.0,0.0
6,6,6,2228.0,2228.0,0.0
7,7,7,64.0,64.0,0.0
8,8,8,2531.0,2530.0,1.0
9,9,9,190.0,190.0,0.0


In [65]:
#Check lottype domain
unique_lottype = df.lottype.tolist()
lottype_domain = list(get_domain('lottype', 'pluto_domains.json').keys())

In [66]:
# in domain not in pluto:
[i for i in lottype_domain if i not in unique_lottype]

[]

In [67]:
# in pluto not in domain:
[i for i in unique_lottype if i not in lottype_domain]

[None, None]

### Basement code

In [68]:
sql = f'''
    WITH newcount as (
    SELECT bsmtcode, count(*)
    FROM {pluto}
    GROUP BY bsmtcode),
    prevcount as (
    SELECT bsmtcode, count(*)
    FROM {plutoprev}
    GROUP BY bsmtcode)
    SELECT a.bsmtcode, b.bsmtcode as bsmtcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bsmtcode = b.bsmtcode
    ORDER BY a.bsmtcode;
'''

In [69]:
df = pd.read_sql(sql=sql, con=con)

In [70]:
df

,bsmtcode,bsmtcodeprev,countnew,countprev,diff
0,0,0,74507.0,74508.0,-1.0
1,1,1,153674.0,153697.0,-23.0
2,2,2,495893.0,495979.0,-86.0
3,3,3,1499.0,1498.0,1.0
4,4,4,4704.0,4705.0,-1.0
5,5,5,128709.0,128599.0,110.0
6,None,None,616.0,NaN,NaN
7,None,None,NaN,473.0,NaN


In [71]:
#Check bsmtcode domain
unique_bsmtcode = df.bsmtcode.tolist()
bsmtcode_domain = list(get_domain('bsmtcode', 'pluto_domains.json').keys())

In [72]:
# in domain not in pluto:
[i for i in bsmtcode_domain if i not in unique_bsmtcode]

[]

In [73]:
# in pluto not in domain:
[i for i in unique_bsmtcode if i not in bsmtcode_domain]

[None, None]

### PLUTO map id

In [74]:
sql = f'''
    WITH newcount as (
    SELECT plutomapid, count(*)
    FROM {pluto}
    GROUP BY plutomapid),
    prevcount as (
    SELECT plutomapid, count(*)
    FROM {plutoprev}
    GROUP BY plutomapid)
    SELECT a.plutomapid, b.plutomapid as plutomapidprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.plutomapid = b.plutomapid
    ORDER BY a.plutomapid;
'''

In [75]:
df = pd.read_sql(sql=sql, con=con)

In [76]:
df

,plutomapid,plutomapidprev,countnew,countprev,diff
0,1,1,856681.0,856861.0,-180.0
1,2,2,2095.0,1915.0,180.0
2,3,3,612.0,469.0,143.0
3,None,None,214.0,NaN,NaN
4,None,4,NaN,210.0,NaN
5,None,5,NaN,4.0,NaN


In [77]:
#Check plutomapid domain
unique_plutomapid = df.plutomapid.tolist()
plutomapid_domain = list(get_domain('plutomapid', 'pluto_domains.json').keys())

In [78]:
# in domain not in pluto:
[i for i in plutomapid_domain if i not in unique_plutomapid]

['4', '5']

In [79]:
# in pluto not in domain:
[i for i in unique_plutomapid if i not in plutomapid_domain]

[None, None, None]

#### done